In [3]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from llama_index import VectorStoreIndex, get_response_synthesizer
from llama_index.schema import Document


In [5]:
from llama_index import load_index_from_storage
from llama_index import StorageContext, ServiceContext


storage_context = StorageContext.from_defaults(persist_dir="research_paper_index")
index = load_index_from_storage(storage_context, index_id="graph_of_thoughts")

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Data Agents

#### basic function calling

In [6]:
from llama_index.tools import FunctionTool
from llama_index.llms import OpenAI
from llama_index.agent import OpenAIAgent, ReActAgent


In [3]:
#define a simple tool
def multiply(a:int, b:int) -> int:
    """multiply 2 numbers and return the integer result"""
    return a*b

#it learns from the doc string.
multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [7]:
#initialize llm
llm = OpenAI(model="gpt-3.5-turbo-0613")

- In openai agent, tool selection is baked into it through fine tuning process - (selects tools given description)
- In react agent, tool selection is computed with ReAct process. (open source llms, gpt4 - but requires more llm calls to select tools)

In [5]:
#create data agent = openai or react

react_agent = ReActAgent.from_tools(tools=[multiply_tool],llm=llm, verbose=True)
openai_agent = OpenAIAgent.from_tools(tools=[multiply_tool], llm=llm, verbose=True)

#### query engines as tools

In [8]:
from llama_index.tools import QueryEngineTool, ToolMetadata

In [20]:
#imagine if we have many different indices from which we can query
uber_query_engine = index.as_query_engine(similarity_top_k=3)
lyft_query_engine = index.as_query_engine(similarity_top_k=3)

query_engine_tools = [
    QueryEngineTool(
        query_engine=uber_query_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description="Provides information about Uber financials for year 2021. Use a detailed plain text query as input to the tool."
        )
    ),
    QueryEngineTool(
        query_engine=lyft_query_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description="Provides information about Lyft financials for year 2021. Use a detailed plain text query as input to the tool."
        )

    )
]

react_agent = ReActAgent.from_tools(
    tools=[multiply_tool, *query_engine_tools], 
    llm=llm, 
    verbose=True)

#### Use other Agents as tools

In [ ]:
# query_engine_tools = [
#     QueryEngineTool(
#         query_engine=sql_agent,
#         metadata=ToolMetadata(
#             name="sql_agent", description="Agent that can execute SQL queries."
#         ),
#     ),
#     QueryEngineTool(
#         query_engine=gmail_agent,
#         metadata=ToolMetadata(
#             name="gmail_agent",
#             description="Tool that can send emails on Gmail.",
#         ),
#     ),
# ]

# outer_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

#### Retrieving tools from an index; Query Planning: only oai agents, not react for now.

In [24]:
from llama_index.objects import ObjectIndex, SimpleToolNodeMapping
from llama_index.agent import FnRetrieverOpenAIAgent

all_tools = query_engine_tools + [multiply_tool]
tool_mapping = SimpleToolNodeMapping.from_objects(all_tools)
object_index = ObjectIndex.from_objects(
    all_tools,
    tool_mapping,
    VectorStoreIndex
)
openai_agent = FnRetrieverOpenAIAgent.from_retriever(
    object_index.as_retriever(),
    verbose=True
)

#### Context Retrieval Agent

Perform retrieval before calling tools --> pick better tools

In [32]:
from llama_index.agent import ContextRetrieverOpenAIAgent

# store a list of abbreviations
texts = [
    "Abbreviation: X = Revenue",
    "Abbreviation: YZ = Risk Factors",
    "Abbreviation: Z = Costs",
]
abbreviation_docs = [Document(text=t) for t in texts]
context_index = VectorStoreIndex.from_documents(abbreviation_docs)

# add context agent
context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    all_tools,
    context_index.as_retriever(similarity_top_k=1),
    verbose=True
)
## response = context_agent.chat("what is YZ of march 2022 ?")

#### Query Planning

- infers a pydantic schema for complicated query plan
- can be used in conjunction with context retrieal agent for more complicated situations.

In [35]:
from llama_index.tools import QueryPlanTool

response_synthesizer = get_response_synthesizer()
query_plan_tool = QueryPlanTool.from_defaults(
    query_engine_tools=query_engine_tools,
    response_synthesizer=response_synthesizer
)

openai_agent = OpenAIAgent.from_tools(
    tools=[query_plan_tool],
    max_function_calls=10,
    llm=OpenAI(temperature=0, model="gpt-4-0613")

)

In [9]:
from pydantic import BaseModel

class User(BaseModel):
    """
    When you create an instance of User with user_data, Pydantic validates the data according to the schema, 
    ensuring that name is a string, age is an integer, etc. 
    If the data doesn't match the schema, it raises an error.
    """
    name: str
    age: int
    is_active: bool = True

# Example usage
user_data = {'name': 'Alice', 'age': 30}
user = User(**user_data)
print(user)

name='Alice' age=30 is_active=True


Do all topics in Model Guide

- utility tools - augment the capabilities of existing tools
- check OnDemandToolLoader, and LoadAndSearchToolSpec

#### Tool Specs

In [38]:
# from llama_index import BaseToolSpec
# # class MyToolsSpec()

#### External tools from llamahub

In [44]:
# from llama_hub import YouubeTranscriptReader
# 
# 

In [47]:
from llama_hub.youtube_transcript import YoutubeTranscriptReader
loader = YoutubeTranscriptReader()
transcript_docs = loader.load_data(ytlinks=["https://www.youtube.com/watch?v=FNXIeEIu6LM"])

In [50]:
sadhguru_index = VectorStoreIndex.from_documents(transcript_docs)

In [57]:
sadhguru_query_engine = sadhguru_index.as_query_engine(response_mode="tree_summarize")

In [58]:
response = sadhguru_query_engine.query("what is sadhguru trying to tell ?")

In [63]:
from pprint import pprint
pprint(response.response)

('Sadhguru is trying to convey the idea that instead of trying to release or '
 'suppress emotions such as anger, lust, or hatred, one should learn to '
 'transform and redirect that energy in the right direction. He emphasizes the '
 'importance of not killing the energy but rather finding the right pathway '
 "for it. Sadhguru also mentions that increasing one's energy levels through "
 'practices and methods can help in this process. Additionally, he suggests '
 "that having a guru in one's life can provide guidance on where the energy "
 'should go. Ultimately, the goal is to accumulate enough energy to burst into '
 'a different dimension or a higher state of being.')


## Retriever

In [66]:
retriever = sadhguru_index.as_retriever()
nodes = retriever.retrieve("How to avoid anger ?")

In [69]:
for n in nodes:
    print(n.text)

[Music]
lust is great energy keep it inside cook
[Music]
yourself don't try to kill that energy
redirect it in the right
direction you have to burst into the
next
[Music]
damage
this is a big difference that you see in
the understanding of Life between east
and west east west has always talked
about
release everything you must release
it your psychological situations you
must find release either in somebody or
something so the Easter way of life is
always don't
release whether it's your anger or your
hatred or your lust whatever keep it
inside cook
yourself if you don't allow it any
release it'll sh
up this is
your don't allow you to release anyway
plug all the
holes if you plug all the holes it will
only go up so this is the way
of this is where these philosophies are
totally diametrical to other ways what
they're saying is
whatever you may do whether you get
angry or you lustful or hatred the
tremendous energy isn't it anger is
great energy isn't
it hatred is great energy lust is gre

For each type of index, there are specific retriever modes with specific purpose.

## Response Synthesizers

With structured_answer_filtering set to True, our refine module is able to filter out any input nodes that are not relevant to the question being asked. This is particularly useful for RAG-based Q&A systems that involve retrieving chunks of text from external vector store for a given user query.

In [75]:
from llama_index.schema import Node
from llama_index.response_synthesizers import ResponseMode, get_response_synthesizer

response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.COMPACT,
    structured_answer_filtering=True #response_mode needs to be compact or refine
)
# same as response_synthesizer = get_response_synthesizer(response_mode="compact", structured_answer_filtering=True)

# response = response_synthesizer.synthesize(
#     "query text",
#     nodes = nodes
# )

In [72]:
response.response

"I'm sorry, but I cannot answer the query as there is no specific query text provided in the context information."

In [73]:
query_engine = index.as_query_engine(response_synthesizer=response_synthesizer)
# query_engine.query(ask someting)

**Response Modes:**

    - refine
    - compact
    - tree_summarize
    - simple_summarize
    - no_text
    - accumulate
    - compact_accumulate

#### custom response synthesizers by inheriting from llama_index.response_synthesizers.base.BaseSynthesizer

#### Custom Prompt Templates (with additional variables)

In [76]:
from llama_index import PromptTemplate
from llama_index.response_synthesizers import TreeSummarize

qa_prompt_tmpl = (
    "Context information is below.\n"
    "-----------------------------\n"
    "{context_str}\n"
    "-----------------------------\n"
    "Given the context information and not prior knowledge,"
    "answer the query.\n"
    "Please also write the answer in the tone of {tone_name}.\n"
    "Query: {query_str}\n"
    "Answer: "
)

qa_prompt = PromptTemplate(qa_prompt_tmpl)

summarizer = TreeSummarize(verbose=True, summary_template=qa_prompt)

# response = summarizer.get_response("query", [text], tone_name="a shakespeare play")

In [ ]:
response_synthesizer = get_response_synthesizer(
    response_mode="",
    service_context=service_context,
    text_qa_template=text_qa_tmpl,
    refine_qa_template=refine_qa_tmpl,
    use_async=False,
    streaming=False
)

# response_synthesizer.synthesize() -> for synchronous
# await response_synthesizer.asynthesize() -> for async

#### Pydantic Tree Summarize

In [77]:
from typing import List
from llama_index.types import BaseModel


class Biography(BaseModel):
    """Data model for a biography."""
    name: str
    best_known_for: List[str]
    extra_info: str

summarizer = TreeSummarize(
    verbose=True,
    summary_template=qa_prompt,
    output_cls=Biography
)

In [ ]:
# response = summarizer.get_response(
#     "who is Paul Graham?", [text], tone_name="a business memo"
# )

## Router

It is basically something that chooses an option from multiple choices. The choosen option is the route it will take.

In [ ]:
#do not run

from llama_index.query_engine.router_query_engine import RouterQueryEngine
from llama_index.selectors.pydantic_selectors import PydanticSingleSelector
from llama_index.tools.query_engine import QueryEngineTool

list_query_engine = "some query engine"
vector_query_engine = "some other query engine"

list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description="Useful for summarizing questions related to data source."
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context related to data source."
)

query_engine = RouterQueryEngine(
        selector=PydanticSingleSelector.from_defaults(),
        query_engine_tools=[
            list_tool,
            vector_tool
        ]
)

# query_engine.query("something")